# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 19 2022 8:22AM,242172,10,SO94315,"Senseonics, Incorporated",Released
1,Jul 19 2022 8:14AM,242170,10,0085883808,ISDIN Corporation,Released
2,Jul 19 2022 8:14AM,242170,10,0085883842,ISDIN Corporation,Released
3,Jul 19 2022 8:14AM,242170,10,0085883846,ISDIN Corporation,Released
4,Jul 19 2022 8:14AM,242170,10,0085883860,ISDIN Corporation,Released
5,Jul 19 2022 8:14AM,242170,10,0085883891,ISDIN Corporation,Released
6,Jul 19 2022 8:14AM,242170,10,0085883881,ISDIN Corporation,Released
7,Jul 19 2022 8:14AM,242170,10,0085883915,ISDIN Corporation,Released
8,Jul 19 2022 8:14AM,242170,10,0085883841,ISDIN Corporation,Released
9,Jul 19 2022 8:14AM,242170,10,0085883845,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,242168,Released,5
47,242169,Released,1
48,242170,Released,25
49,242171,Released,1
50,242172,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242168,NaN,NaN,5.0
242169,NaN,NaN,1.0
242170,NaN,NaN,25.0
242171,NaN,NaN,1.0
242172,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4.0,3.0,5.0
241972,2.0,11.0,1.0
242007,0.0,0.0,1.0
242008,0.0,0.0,1.0
242009,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4,3,5
241972,2,11,1
242007,0,0,1
242008,0,0,1
242009,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,3,5
1,241972,2,11,1
2,242007,0,0,1
3,242008,0,0,1
4,242009,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,3,5
1,241972,2,11,1
2,242007,,,1
3,242008,,,1
4,242009,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 19 2022 8:22AM,242172,10,"Senseonics, Incorporated"
1,Jul 19 2022 8:14AM,242170,10,ISDIN Corporation
26,Jul 19 2022 8:12AM,242171,10,Beach Products Inc
27,Jul 19 2022 8:08AM,242169,10,Emerginnova
28,Jul 19 2022 7:31AM,242168,19,ACG North America LLC
33,Jul 19 2022 7:08AM,242167,19,"AdvaGen Pharma, Ltd"
35,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc."
36,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc."
37,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc."
38,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 19 2022 8:22AM,242172,10,"Senseonics, Incorporated",,,1
1,Jul 19 2022 8:14AM,242170,10,ISDIN Corporation,,,25
2,Jul 19 2022 8:12AM,242171,10,Beach Products Inc,,,1
3,Jul 19 2022 8:08AM,242169,10,Emerginnova,,,1
4,Jul 19 2022 7:31AM,242168,19,ACG North America LLC,,,5
5,Jul 19 2022 7:08AM,242167,19,"AdvaGen Pharma, Ltd",,,2
6,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc.",,,1
7,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc.",,,1
8,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc.",,,1
9,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 8:22AM,242172,10,"Senseonics, Incorporated",1,,
1,Jul 19 2022 8:14AM,242170,10,ISDIN Corporation,25,,
2,Jul 19 2022 8:12AM,242171,10,Beach Products Inc,1,,
3,Jul 19 2022 8:08AM,242169,10,Emerginnova,1,,
4,Jul 19 2022 7:31AM,242168,19,ACG North America LLC,5,,
5,Jul 19 2022 7:08AM,242167,19,"AdvaGen Pharma, Ltd",2,,
6,Jul 18 2022 5:20PM,242161,21,"NBTY Global, Inc.",1,,
7,Jul 18 2022 5:14PM,242160,21,"NBTY Global, Inc.",1,,
8,Jul 18 2022 4:37PM,242159,20,"ACI Healthcare USA, Inc.",1,,
9,Jul 18 2022 4:12PM,242158,16,Sartorius Lab Products and Service,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 8:22AM,242172,10,"Senseonics, Incorporated",1,,
1,Jul 19 2022 8:14AM,242170,10,ISDIN Corporation,25,,
2,Jul 19 2022 8:12AM,242171,10,Beach Products Inc,1,,
3,Jul 19 2022 8:08AM,242169,10,Emerginnova,1,,
4,Jul 19 2022 7:31AM,242168,19,ACG North America LLC,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 8:22AM,242172,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jul 19 2022 8:14AM,242170,10,ISDIN Corporation,25.0,NaN,NaN
2,Jul 19 2022 8:12AM,242171,10,Beach Products Inc,1.0,NaN,NaN
3,Jul 19 2022 8:08AM,242169,10,Emerginnova,1.0,NaN,NaN
4,Jul 19 2022 7:31AM,242168,19,ACG North America LLC,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 8:22AM,242172,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jul 19 2022 8:14AM,242170,10,ISDIN Corporation,25.0,0.0,0.0
2,Jul 19 2022 8:12AM,242171,10,Beach Products Inc,1.0,0.0,0.0
3,Jul 19 2022 8:08AM,242169,10,Emerginnova,1.0,0.0,0.0
4,Jul 19 2022 7:31AM,242168,19,ACG North America LLC,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2420962,52.0,7.0,26.0
102,242008,1.0,0.0,0.0
16,484315,5.0,0.0,0.0
19,1452849,18.0,0.0,15.0
20,1936441,136.0,25.0,8.0
21,3631740,15.0,0.0,0.0
22,242117,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2420962,52.0,7.0,26.0
1,102,242008,1.0,0.0,0.0
2,16,484315,5.0,0.0,0.0
3,19,1452849,18.0,0.0,15.0
4,20,1936441,136.0,25.0,8.0
5,21,3631740,15.0,0.0,0.0
6,22,242117,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,52.0,7.0,26.0
1,102,1.0,0.0,0.0
2,16,5.0,0.0,0.0
3,19,18.0,0.0,15.0
4,20,136.0,25.0,8.0
5,21,15.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,102,Released,1.0
2,16,Released,5.0
3,19,Released,18.0
4,20,Released,136.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,19,20,21,22
Status,,,,,,,
Completed,26.0,0.0,0.0,15.0,8.0,0.0,0.0
Executing,7.0,0.0,0.0,0.0,25.0,0.0,1.0
Released,52.0,1.0,5.0,18.0,136.0,15.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,19,20,21,22
0,Completed,26.0,0.0,0.0,15.0,8.0,0.0,0.0
1,Executing,7.0,0.0,0.0,0.0,25.0,0.0,1.0
2,Released,52.0,1.0,5.0,18.0,136.0,15.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,19,20,21,22
0,Completed,26.0,0.0,0.0,15.0,8.0,0.0,0.0
1,Executing,7.0,0.0,0.0,0.0,25.0,0.0,1.0
2,Released,52.0,1.0,5.0,18.0,136.0,15.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()